# Python Descriptors and How to Use them | Towards Data Science  

## Artemis Nika8-9 minutes 14/03/2022  


https://towardsdatascience.com/python-descriptors-and-how-to-use-them-5167d506af84

In [9]:
class ExplainDescriptors:

    def __init__(self):
        print('__init__')

    def __set_name__(self, owner, name):
        print(f'__set_name__(owner={owner}, name={name})')
        self.name = name

    def __get__(self, instance, owner=None):
        print(f'__get__(instance={instance}, owner={owner})')
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value): 
        print(f'__set__(instance={instance}, value={value})')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        print(f'__delete__(instance={instance})')
        del instance.__dict__[self.name]

da definire

In [10]:
class SomeClass:
    a = ExplainDescriptors()

__init__
__set_name__(owner=<class '__main__.SomeClass'>, name=a)


In [11]:

someObject = SomeClass()

In [12]:
someObject.a = 10

__set__(instance=<__main__.SomeClass object at 0x000001A74CCD05B0>, value=10)


In [13]:

del someObject.a

__delete__(instance=<__main__.SomeClass object at 0x000001A74CCD05B0>)


In [14]:

someObject.a

__get__(instance=<__main__.SomeClass object at 0x000001A74CCD05B0>, owner=<class '__main__.SomeClass'>)


Let’s have a look at a more practical example (without print statements this time)!

In [15]:
class ReadOnlyAttribute:

    def __init__(self, value):
        self.value = value

    def __set_name__(self, owner, name):
        self.name = name 

    def __get__(self, instance, owner=None):
        return self.value

    def __set__(self, instance, value):
        msg = '{} is a read-only attribute'.format(self.name)
        raise AttributeError(msg)

In [16]:
class StubbornPikachu:
  speak = ReadOnlyAttribute('pika pika')

In the example above, we have created a read-only descriptor and assigned it to the attribute speak. We are not storing anything in instance as we want all Pikachu objects to say the same thing — they all share the same descriptor.

Any attempts to change what our StubbornPikachu says will be met with failure (or more specifically an AttributeError).

In [5]:
class DataDescriptor:
    def __set_name__(self, owner, name):
        self.name = name
    def __get__(self, instance, owner=None):
        default_value = '{} not defined'.format(self.name)
        print('In __get__ of {}'.format(self))
        return instance.__dict__.get(self.name, default_value)
    def __set__(self, instance, value):
        print('In __set__ of {}'.format(self))
        instance.__dict__[self.name] = value

class NonDataDescriptor:
    def __set_name__(self, owner, name):
        self.name = name
    def __get__(self, instance, owner=None):
        default_value = '{} not defined'.format(self.name)
        print('In __get__ of {}'.format(self))
        return instance.__dict__.get(self.name, default_value)

class Example:
    a = DataDescriptor()
    b = NonDataDescriptor()

example = Example()

In the code above we have created two descriptors and assigned them to attributes a and b for class Example. 
They are identical except that *DataDescriptor has the \_\_set\_\_ method.*

In [6]:
example.__dict__

{}

we haven’t defined a and b for example — and hence example.__dict__ is empty  
both attributes are accessed via their descriptors' \_\_get\_\_ methods

In [7]:
example.a

In __get__ of <__main__.DataDescriptor object at 0x0000021024296BE0>


'a not defined'

In [8]:
example.b

In __get__ of <__main__.NonDataDescriptor object at 0x0000021024296CD0>


'b not defined'

In [14]:
# we set them to some values.
example.__dict__['a'] = 'AAA'
example.__dict__['b'] = 'BBB'
example.__dict__

{'a': 'AAA', 'b': 'BBB'}

After ‘b’ appears in example.\_\_dict\_\_ the \_\_get\_\_ method of the NonDataDescriptor assigned to b is no longer used. 

In [15]:
print(example.a)
example.b

In __get__ of <__main__.DataDescriptor object at 0x0000021024296BE0>
AAA


'BBB'

After ‘b’ appears in example.\_\_dict\_\_ the \_\_get\_\_ method of the NonDataDescriptor assigned to b is no longer used.  
Accessing a still uses the \_\_get\_\_ method as data descriptors take priority over whatever is in \_\_dict\_\_.  
To access b , however, we go straight to example.\_\_dict\_\_ as that is checked before non-data descriptors.


class BoundedNumber below is a data descriptor that checks that the value assigned to the attribute is between two numbers

In [16]:
class BoundedNumber:
    def __init__(self, min_val, max_val):
        self.min_val = min_val
        self.max_val = max_val
    def __set_name__(self, owner, name):
        self.name = name
    def __set__(self, instance, value):
        if self.min_val > value or value > self.max_val:
            msg = '{} takes values between {} and {}'.format(
                self.name, 
                self.min_val, 
                self.max_val,
            )
            raise ValueError(msg)
        instance.__dict__[self.name] = value
    def __get__(self, instance, owner=None):
        return instance.__dict__[self.name]

In [17]:
class Person:
    age = BoundedNumber(1, 120)
    weight = BoundedNumber(1, 250)
    height = BoundedNumber(1, 230)
    def __init__(self, name, age, weight, height):
        self.name = name
        self.age = age
        self.weight = weight
        self.height = height


In [18]:
p1 = Person('John',  age=20,  weight = 80, height = 170) 
p1 = Person('Maria', age=432, weight = 60, height = 160)

ValueError: age takes values between 1 and 120

In [13]:
import numpy as np
image_width = 4
image_height = 4
colours = 3
image = np.zeros((image_width, image_height,3))
resized_image = image.reshape(image.shape[0]*image.shape[1]*image.shape[2],-1).T
print("reshaped",resized_image.shape)
print("transposed",resized_image.T.shape)

reshaped (1, 48)
transposed (48, 1)
